# check eventdet oof df

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [4]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
# train_event_df = train_event_df[train_event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
train_event_df = train_event_df.reset_index(drop=True)
display(train_event_df.head())

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [5]:
import sys
sys.path.append("/kaggle/src/dss_utils")

from dss_metrics import score

In [6]:
event_df = train_event_df[train_event_df["step"].notnull()].copy()

# 1st stage

In [7]:
fold_num = 1

In [8]:
output_dir = os.path.join("/kaggle", "working", "_oof")
exp_name = "exp008_meanstd_nonavepool"
print("load oof df")
oof_df = pd.DataFrame()
for i in range(fold_num):
    print("fold", i)
    df = pd.read_parquet(os.path.join(output_dir, exp_name, f"oof_df_fold{i}.parquet"))
    for col in df.columns:
        # 64bit float -> 16bit float
        if df[col].dtype == np.float64:
            df[col] = df[col].astype(np.float16)
        # 64bit int -> 16bit int
        elif df[col].dtype == np.int64:
            df[col] = df[col].astype(np.int16)

    oof_df = pd.concat([oof_df, df], axis=0)

load oof df
fold 0


In [9]:
# postprocess_fn
# series_idでgroupbyして、class_predに対して対象の列のデータから前のN個の列までのデータの平均をとる
import torch
import torch.nn as nn

# 1step 0.5secで30minなら60*30=1800step
def postprocess_fn(df, N=1800, maxpool_kernel_size=101, maxpool_stride=1):
    df = df.copy()
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x.rolling(N, min_periods=1).mean())
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1])
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]

    # 入力サイズと出力サイズが一致するようにpaddingを調整
    maxpool_padding = int((maxpool_kernel_size - maxpool_stride) / 2)
    # maxpoolしてピーク検出
    max_pooling = nn.MaxPool1d(maxpool_kernel_size, stride=maxpool_stride, padding=maxpool_padding)
    event_pred = df["event_pred"].values
    event_pred = torch.tensor(event_pred).unsqueeze(0)
    pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
    event_pred = event_pred.squeeze(0).numpy()
    # peakのところだけ残すmaskを作成
    peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
    peak_event_pred = event_pred * peak_event_pred_mask
    df["event_pred"] = peak_event_pred
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)
    return df


def make_submission_df(df, threshold=0.1):
    df = df[["series_id", "step", "event_pred"]].copy()
    # thresholdより大きいときは1,-thresholdより小さいときは-1,それ以外は0
    df["event"] = df["event_pred"].apply(lambda x: 1 if x > threshold else -1 if x < -threshold else 0)
    df = df[df["event"] != 0].copy()
    df["event"] = df["event"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    return df


In [10]:
df = postprocess_fn(oof_df, N=301, maxpool_kernel_size=61)
sub_df = make_submission_df(df, threshold=0.01)
event_ = event_df[event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
print(score(event_, sub_df))

0.6534456351810006


# 2nd stage

In [11]:
output_dir = os.path.join("/kaggle", "working", "_oof")
# exp_name = "event_det_debug"
exp_name = "debug_secondstg"
print("load oof df")
oof_df = pd.DataFrame()
# for i in range(5):
for i in range(fold_num):
    print("fold", i)
    df = pd.read_parquet(os.path.join(output_dir, exp_name, f"oof_df_fold{i}.parquet"))
    for col in df.columns:
        # 64bit float -> 16bit float
        if df[col].dtype == np.float64:
            df[col] = df[col].astype(np.float16)
        # 64bit int -> 16bit int
        elif df[col].dtype == np.int64:
            df[col] = df[col].astype(np.int16)

    oof_df = pd.concat([oof_df, df], axis=0)

load oof df
fold 0


In [12]:
oof_df.columns

Index(['series_id', 'step', 'anglez', 'enmo', 'event', 'event_onset',
       'event_wakeup', 'date', 'series_date_key', 'series_date_key_str',
       'fold', 'class_pred', 'event_pred', 'event_target', 'onset_pred',
       'wakeup_pred', 'onset_target', 'wakeup_target'],
      dtype='object')

In [13]:
oof_df["step"] = oof_df["step"].astype(np.float64)

In [14]:
import torch
import torch.nn as nn

def make_submission_from_eventdf(df, threshold=0.1):
    df = df[["series_id", "step", "onset_pred", "wakeup_pred"]].copy()
    df["step"] = df["step"].astype(np.float64)
    max_pool = nn.MaxPool1d(11, stride=1, padding=5)
    onset_pred = max_pool(torch.tensor(df["onset_pred"].values).unsqueeze(0)).squeeze(0).numpy()
    wakeup_pred = max_pool(torch.tensor(df["wakeup_pred"].values).unsqueeze(0)).squeeze(0).numpy()
    peak_mask = (onset_pred == df["onset_pred"].values)
    df["onset_pred"] = peak_mask * df["onset_pred"].values
    peak_mask = (wakeup_pred == df["wakeup_pred"].values)
    df["wakeup_pred"] = peak_mask * df["wakeup_pred"].values    
    # onset_predが大きい場合-onset_predの値を入力し、wakeup_predが大きい場合wakeup_predの値を入力する
    df["event_pred"] = np.where(
        df["onset_pred"].values > df["wakeup_pred"].values,
        -df["onset_pred"].values,
        df["wakeup_pred"].values,
    )
    # event_predがthreshold以上の場合、wakeup_predが大きい場合はwakeup、onset_predが大きい場合はonsetとする
    df["event_score"] = df["event_pred"].apply(
        lambda x: 1 if x > threshold else -1 if x < -threshold else 0
    )
    df = df[df["event_score"] != 0].copy()
    df["event"] = df["event_score"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    df = df.drop(["event_pred", "onset_pred", "wakeup_pred", "event_score"], axis=1)
    return df

In [15]:
sub_df_2nd_stage = make_submission_from_eventdf(oof_df, threshold=0.1)

In [16]:
event_series_df = event_df[event_df["series_id"].isin(oof_df["series_id"].unique())].copy()

In [17]:
score(event_series_df, sub_df_2nd_stage)

0.2609323275555059

In [18]:
sub_df_1an2_stg = pd.concat([sub_df, sub_df_2nd_stage], axis=0)
event_ = event_df[event_df["series_id"].isin(oof_df["series_id"].unique())].copy()

In [19]:
score(event_, sub_df_1an2_stg)

0.6534456351810006